In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-devil/cleaned_data(defoult).csv


In [41]:
# Импорт основных библиотек
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error, r2_score
)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

sns.set(style="whitegrid")

RANDOM_STATE = 42


In [42]:
# Загружаем очищенный датасет кредитного скоринга
df = pd.read_csv('/kaggle/input/data-devil/cleaned_data(defoult).csv')

df.shape, df.head()


((30000, 24),
    LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
 0    20000.0    2          2         1   24      2      2     -1     -1   
 1   120000.0    2          2         2   26     -1      2      0      0   
 2    90000.0    2          2         2   34      0      0      0      0   
 3    50000.0    2          2         1   37      0      0      0      0   
 4    50000.0    1          2         1   57     -1      0     -1      0   
 
    PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
 0     -2  ...        0.0        0.0        0.0       0.0     689.0       0.0   
 1      0  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
 2      0  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
 3      0  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
 4      0  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   
 
    PAY_AMT4  PAY_AMT5  PAY_AMT6  defaul

In [43]:
df['default.payment.next.month'].value_counts(normalize=True)

default.payment.next.month
0    0.7788
1    0.2212
Name: proportion, dtype: float64

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   LIMIT_BAL                   30000 non-null  float64
 1   SEX                         30000 non-null  int64  
 2   EDUCATION                   30000 non-null  int64  
 3   MARRIAGE                    30000 non-null  int64  
 4   AGE                         30000 non-null  int64  
 5   PAY_0                       30000 non-null  int64  
 6   PAY_2                       30000 non-null  int64  
 7   PAY_3                       30000 non-null  int64  
 8   PAY_4                       30000 non-null  int64  
 9   PAY_5                       30000 non-null  int64  
 10  PAY_6                       30000 non-null  int64  
 11  BILL_AMT1                   30000 non-null  float64
 12  BILL_AMT2                   30000 non-null  float64
 13  BILL_AMT3                   300

In [45]:
from sklearn.compose import ColumnTransformer
# Целевая переменная и X
y = df['default.payment.next.month']
X = df.drop(columns=['default.payment.next.month'])

# Категориальные признаки по смыслу
categorical = ['SEX', 'EDUCATION', 'MARRIAGE']

# Остальные считаем числовыми
numerical = [col for col in X.columns if col not in categorical]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical),
        ('cat', OneHotEncoder(drop='first'), categorical)
    ]
)

# Преобразуем X
X_processed = preprocessor.fit_transform(X)
X_processed.shape

(30000, 30)

In [46]:
# Разбиваем данные на обучающую и тестовую выборки.
# Стратификация важна, т.к. классы несбалансированы (default ~22%).
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y
)

X_train.shape, X_test.shape

((24000, 30), (6000, 30))

In [47]:
# Базовый случайный лес без подбора параметров.
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

baseline_rf = RandomForestClassifier(
    random_state=RANDOM_STATE,
    n_estimators=100
)

baseline_rf.fit(X_train, y_train)
y_pred_base = baseline_rf.predict(X_test)
y_prob_base = baseline_rf.predict_proba(X_test)[:, 1]

baseline_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred_base),
    "Precision": precision_score(y_test, y_pred_base),
    "Recall": recall_score(y_test, y_pred_base),
    "F1": f1_score(y_test, y_pred_base),
    "ROC-AUC": roc_auc_score(y_test, y_prob_base)
}

baseline_metrics


{'Accuracy': 0.8133333333333334,
 'Precision': 0.6349413298565841,
 'Recall': 0.36699321778447624,
 'F1': 0.46513849092645654,
 'ROC-AUC': 0.7538538100918375}

In [48]:
#GridSearchCV для Random Forest

from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [100],
    "max_depth": [6, 9, None],
    "min_samples_split": [2, 10],
    "min_samples_leaf": [1, 5],
    "class_weight": ["balanced"]
}

rf = RandomForestClassifier(random_state=RANDOM_STATE)

grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="f1",
    n_jobs=-1,
    cv=5,
    verbose=1
)

grid_rf.fit(X_train, y_train)
grid_rf.best_params_


Fitting 5 folds for each of 12 candidates, totalling 60 fits


{'class_weight': 'balanced',
 'max_depth': 9,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [49]:
#После подбора: обучаем улучшенную модель

best_rf = grid_rf.best_estimator_

y_pred_improved = best_rf.predict(X_test)
y_prob_improved = best_rf.predict_proba(X_test)[:, 1]

improved_metrics = {
    "Accuracy": accuracy_score(y_test, y_pred_improved),
    "Precision": precision_score(y_test, y_pred_improved),
    "Recall": recall_score(y_test, y_pred_improved),
    "F1": f1_score(y_test, y_pred_improved),
    "ROC-AUC": roc_auc_score(y_test, y_prob_improved)
}

improved_metrics

{'Accuracy': 0.7881666666666667,
 'Precision': 0.5192837465564738,
 'Recall': 0.5681989449886963,
 'F1': 0.5426412378553436,
 'ROC-AUC': 0.7753609013668767}

In [50]:
#Сравнение результатов

import pandas as pd

pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC-AUC"],
    "Baseline RF": list(baseline_metrics.values()),
    "Improved RF": list(improved_metrics.values())
})


,Metric,Baseline RF,Improved RF
0,Accuracy,0.813333,0.788167
1,Precision,0.634941,0.519284
2,Recall,0.366993,0.568199
3,F1,0.465138,0.542641
4,ROC-AUC,0.753854,0.775361


In [51]:
# Самописное дерево для RandomForest

import numpy as np

class SimpleDecisionTreeClassifier:
    def __init__(self, max_depth=None, min_samples_leaf=1, feature_subsample_size=None):
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.feature_subsample_size = feature_subsample_size
        self.tree = None

    def _gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        p = counts / len(y)
        return 1 - np.sum(p**2)

    def _best_split(self, X, y, features):
        best_feat, best_thresh = None, None
        best_gini = 1e9
        n_samples = len(y)

        for feat in features:
            thresholds = np.unique(X[:, feat])

            for t in thresholds:
                left = X[:, feat] <= t
                right = ~left

                if left.sum() < self.min_samples_leaf or right.sum() < self.min_samples_leaf:
                    continue

                g = (
                    self._gini(y[left]) * left.sum() +
                    self._gini(y[right]) * right.sum()
                ) / n_samples

                if g < best_gini:
                    best_gini = g
                    best_feat = feat
                    best_thresh = t

        return best_feat, best_thresh

    def _build(self, X, y, depth):
        if len(np.unique(y)) == 1:
            return {"leaf": True, "class": y[0]}

        if self.max_depth is not None and depth >= self.max_depth:
            return {"leaf": True, "class": np.bincount(y).argmax()}

        # выбираем подмножество признаков (bagging по признакам)
        n_features = X.shape[1]
        k = self.feature_subsample_size or int(np.sqrt(n_features))
        features = np.random.choice(n_features, k, replace=False)

        feat, thresh = self._best_split(X, y, features)

        if feat is None:
            return {"leaf": True, "class": np.bincount(y).argmax()}

        left = X[:, feat] <= thresh
        right = ~left

        return {
            "leaf": False,
            "feature": feat,
            "threshold": thresh,
            "left": self._build(X[left], y[left], depth+1),
            "right": self._build(X[right], y[right], depth+1)
        }

    def fit(self, X, y):
        self.tree = self._build(X, y, 0)
        return self

    def _predict_one(self, node, x):
        if node["leaf"]:
            return node["class"]

        if x[node["feature"]] <= node["threshold"]:
            return self._predict_one(node["left"], x)
        else:
            return self._predict_one(node["right"], x)

    def predict(self, X):
        return np.array([self._predict_one(self.tree, x) for x in X])


In [52]:
# Самописный RandomForestClassifier

class MyRandomForestClassifier:
    def __init__(
        self,
        n_estimators=10,
        max_depth=None,
        min_samples_leaf=1,
        feature_subsample_size=None,
        bootstrap=True
    ):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.feature_subsample_size = feature_subsample_size
        self.bootstrap = bootstrap
        self.trees = []

    def fit(self, X, y):
        n_samples = X.shape[0]

        for _ in range(self.n_estimators):
            # бутстрэп выборка
            if self.bootstrap:
                idx = np.random.choice(n_samples, n_samples, replace=True)
                X_sample = X[idx]
                y_sample = y[idx]
            else:
                X_sample, y_sample = X, y

            tree = SimpleDecisionTreeClassifier(
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                feature_subsample_size=self.feature_subsample_size
            )
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

        return self

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        # голосование
        final_preds = np.round(predictions.mean(axis=0)).astype(int)
        return final_preds


In [53]:
def to_dense(X):
    try: 
        return X.toarray()
    except:
        return np.asarray(X)

X_train_np = to_dense(X_train)
X_test_np  = to_dense(X_test)
y_train_np = np.asarray(y_train)
y_test_np  = np.asarray(y_test)

In [55]:
my_rf = MyRandomForestClassifier(
    n_estimators=5,
    max_depth=3,
    min_samples_leaf=5
)

my_rf.fit(X_train_np, y_train_np)
y_pred_my = my_rf.predict(X_test_np)

my_rf_baseline_results = {
    "Accuracy": accuracy_score(y_test_np, y_pred_my),
    "Precision": precision_score(y_test_np, y_pred_my),
    "Recall": recall_score(y_test_np, y_pred_my),
    "F1": f1_score(y_test_np, y_pred_my),
    "ROC-AUC": roc_auc_score(y_test_np, y_pred_my)
}

my_rf_baseline_results


{'Accuracy': 0.804,
 'Precision': 0.6421845574387948,
 'Recall': 0.2569706103993971,
 'F1': 0.3670613562970936,
 'ROC-AUC': 0.6081557524498591}

In [58]:
my_rf_imp = MyRandomForestClassifier(
    n_estimators=8,
    max_depth=5,
    min_samples_leaf=2,
    feature_subsample_size=int(np.sqrt(X_train_np.shape[1]))
)

my_rf_imp.fit(X_train_np, y_train_np)
y_pred_imp = my_rf_imp.predict(X_test_np)

my_rf_improved_results = {
    "Accuracy": accuracy_score(y_test_np, y_pred_imp),
    "Precision": precision_score(y_test_np, y_pred_imp),
    "Recall": recall_score(y_test_np, y_pred_imp),
    "F1": f1_score(y_test_np, y_pred_imp),
    "ROC-AUC": roc_auc_score(y_test_np, y_pred_imp)
}

my_rf_improved_results


{'Accuracy': 0.813,
 'Precision': 0.6716917922948074,
 'Recall': 0.3021853805576488,
 'F1': 0.41683991683991684,
 'ROC-AUC': 0.6301211516526742}

In [63]:
import pandas as pd

df_compare = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1", "ROC-AUC"],
    "sklearn_baseline": [
        0.813333, 0.634941, 0.366993, 0.465138, 0.753854
    ],
    "sklearn_improved": [
        0.788167, 0.519284, 0.568199, 0.542641, 0.775361
    ],
    "my_baseline": [
        my_rf_baseline_results["Accuracy"],
        my_rf_baseline_results["Precision"],
        my_rf_baseline_results["Recall"],
        my_rf_baseline_results["F1"],
        my_rf_baseline_results["ROC-AUC"]
    ],
    "my_improved": [
        my_rf_improved_results["Accuracy"],
        my_rf_improved_results["Precision"],
        my_rf_improved_results["Recall"],
        my_rf_improved_results["F1"],
        my_rf_improved_results["ROC-AUC"]
    ],
})

df_compare


,Metric,sklearn_baseline,sklearn_improved,my_baseline,my_improved
0,Accuracy,0.813333,0.788167,0.804000,0.813000
1,Precision,0.634941,0.519284,0.642185,0.671692
2,Recall,0.366993,0.568199,0.256971,0.302185
3,F1,0.465138,0.542641,0.367061,0.416840
4,ROC-AUC,0.753854,0.775361,0.608156,0.630121
